In [22]:
import pandas as pd
import numpy as np
import re
import pickle

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from catboost import CatBoostRegressor

# -------------------------------
# Función para procesar la columna "planta"
# Convierte cadenas como "1ª", "2ª" o "BAJO" a valores numéricos
def process_planta(valor):
    if pd.isnull(valor):
        return np.nan
    valor = str(valor)
    if "BAJO" or "" in valor.upper():
        return 0
    # Extrae el primer dígito presente en la cadena
    digitos = re.findall(r'\d+', valor)
    if digitos:
        return int(digitos[0])
    return np.nan

# -------------------------------
# Función para cargar el dataset desde un archivo CSV
def load_data(filepath):
    df = pd.read_csv(filepath)
    return df
    
# -------------------------------
# Función para preprocesar el dataframe
def preprocess_data(df):
    # 1. Eliminar columnas descartables
    df = df.drop(columns=['provincia', 'PrecioAnterior', 'descripcion', 'Enlace', 'titulo'])

    # 2. Calcular la media del precio para cada zona
    mean_prices = df.groupby('zona')['PrecioActual'].mean()

    # Ordenar las zonas de menor a mayor media
    sorted_zones = mean_prices.sort_values()

    # Asignar un ranking (0 para la zona con la menor media, y así sucesivamente)
    zona_ranking = {zona: rank for rank, zona in enumerate(sorted_zones.index)}

    # Asignar el ranking como una nueva columna en el DataFrame
    df['zona_rank'] = df['zona'].map(zona_ranking)
    df = df.drop(columns=['zona'])
    
    # 3. Procesar la columna "ascensor": mapear 'S' a 1 y 'N' a 0
    df['ascensor'] = df['ascensor'].map({'S': 1, 'N': 0, "":0, None: 0})
    
    # 4. Procesar la columna "localizacion": rellenar valores nulos con una etiqueta "Desconocido"
    df['localizacion'] = df['localizacion'].fillna('Desconocido')
    
    # 5. Procesar la columna "planta": extraer un valor numérico
    df['planta_processed'] = df['planta'].apply(process_planta)
    df = df.drop(columns=['planta'])
    
    # 6. Procesar la columna "tags": convertir la cadena separada por comas en variables dummy
    df['tags'] = df['tags'].fillna("")
    # Se convierte cada string en una lista de tags (limpiando espacios)
    df['tags_list'] = df['tags'].apply(lambda x: [tag.strip() for tag in x.split(',')] if x != "" else [])
    mlb = MultiLabelBinarizer()
    tags_df = pd.DataFrame(mlb.fit_transform(df['tags_list']),
                           columns=['tag_' + t for t in mlb.classes_],
                           index=df.index)
    # Se concatenan las columnas dummy al dataframe original
    df = pd.concat([df, tags_df], axis=1)
    df = df.drop(columns=['tags', 'tags_list'])
    
    # 7. Codificar variables categóricas: "localizacion"
    # Se utiliza get_dummies y se evita la trampa de la multicolinealidad (drop_first=True)
    df = pd.get_dummies(df, columns=['localizacion'], drop_first=True)

    # 8. Manejar valores nulos: se puede optar por eliminar filas o rellenar con la media/mediana
    # Aquí se opta por eliminar filas con valores nulos
    df = df.dropna()
    
    return df

# -------------------------------
# Función para separar la variable objetivo y las features
def separate_target(df):
    y = df['PrecioActual']
    X = df.drop(columns=['PrecioActual'])
    return X, y

# -------------------------------
# Función para entrenar un modelo con un pipeline y realizar validación cruzada
def train_model(model, X_train, y_train):
    # Se define un pipeline que primero escala las variables y luego entrena el modelo
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    
    # Validación cruzada (5-fold) usando MSE (se usa el negativo MSE para obtener valores positivos)
    cv_scores = cross_val_score(pipeline, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
    cv_mse = -cv_scores.mean()
    
    # Entrenamiento final sobre el conjunto de entrenamiento
    pipeline.fit(X_train, y_train)
    
    return pipeline, cv_mse

# -------------------------------
# Función para optimizar hiperparámetros usando GridSearchCV
def tune_model(pipeline, param_grid, X_train, y_train):
    grid = GridSearchCV(pipeline, param_grid, 
                        cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(X_train, y_train)
    best_pipeline = grid.best_estimator_
    best_cv_mse = -grid.best_score_
    print("Mejores parámetros:", grid.best_params_)
    print("Mejor MSE en CV:", best_cv_mse)
    return best_pipeline, best_cv_mse

# -------------------------------
# Función para evaluar un modelo en el conjunto de test
def evaluate_model(model_pipeline, X_test, y_test):
    y_pred = model_pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, mae, r2

# -------------------------------
# Función principal
def main():
    # 1. Carga de datos
    filepath = './Datos.csv'  # Asegúrate que este archivo esté en el mismo directorio o ajusta la ruta
    df = load_data(filepath)
    
    # 2. Preprocesamiento
    df = preprocess_data(df)
    
    # 3. Separar variable objetivo y variables predictoras
    X, y = separate_target(df)
    
    # 4. División en conjuntos de entrenamiento y test (80%-20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 5. Definición de modelos a evaluar
    models = {
        'LinearRegression': LinearRegression(),
        'RandomForest': RandomForestRegressor(random_state=42),
        'XGBoost': xgb.XGBRegressor(random_state=42, objective='reg:squarederror'),
        'CatBoost': CatBoostRegressor(random_state=42, verbose=0)
    }
    
    # Diccionarios con grids de hiperparámetros para cada modelo
    param_grids = {
        'RandomForest': {
            'model__n_estimators': [50, 100, 150],
            'model__max_depth': [None, 10, 20]
        },
        'XGBoost': {
            'model__n_estimators': [50, 100, 150],
            'model__learning_rate': [0.01, 0.1, 0.2],
            'model__max_depth': [3, 5, 7]
        },
        'CatBoost': {
            'model__iterations': [100, 200],
            'model__learning_rate': [0.01, 0.1],
            'model__depth': [3, 5, 7]
        }
        # Para LinearRegression, normalmente no se optimizan hiperparámetros
    }
    
    results = {}
    model_pipelines = {}
    
    print("Entrenando y evaluando modelos...\n")
    
    # 6. Entrenar y optimizar cada modelo
    for name, model in models.items():
        print(f"Procesando modelo: {name}")
        # Se arma el pipeline base (scaler + modelo)
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', model)
        ])
        
        # Si existen parámetros a optimizar para el modelo, se realiza GridSearchCV
        if name in param_grids:
            best_pipeline, cv_mse = tune_model(pipeline, param_grids[name], X_train, y_train)
        else:
            best_pipeline, cv_mse = train_model(model, X_train, y_train)
        
        test_mse, test_mae, test_r2 = evaluate_model(best_pipeline, X_test, y_test)
        results[name] = {
            'CV_MSE': cv_mse,
            'Test_MSE': test_mse,
            'Test_MAE': test_mae,
            'Test_R2': test_r2
        }
        model_pipelines[name] = best_pipeline
        
        print(f"Modelo: {name}")
        print(f"  MSE (CV): {cv_mse:.3f}")
        print(f"  MSE (Test): {test_mse:.3f}")
        print(f"  MAE (Test): {test_mae:.3f}")
        print(f"  R² (Test): {test_r2:.3f}\n")
    
    # 7. Seleccionar el modelo con el menor MSE en test
    best_model_name = min(results, key=lambda k: results[k]['Test_MSE'])
    best_pipeline = model_pipelines[best_model_name]
    print(f"El mejor modelo según MSE en test es: {best_model_name}")
    
    # 8. Guardar el modelo en un archivo pickle
    with open('best_model.pkl', 'wb') as f:
        pickle.dump(best_pipeline, f)
    print("Modelo guardado en 'best_model.pkl'.")
    
if __name__ == "__main__":
    main()

Entrenando y evaluando modelos...

Procesando modelo: LinearRegression
Modelo: LinearRegression
  MSE (CV): 88009402474175.344
  MSE (Test): 986336213997.817
  MAE (Test): 518933.069
  R² (Test): 0.404

Procesando modelo: RandomForest
Mejores parámetros: {'model__max_depth': None, 'model__n_estimators': 100}
Mejor MSE en CV: 214338741135.64923
Modelo: RandomForest
  MSE (CV): 214338741135.649
  MSE (Test): 308138208440.332
  MAE (Test): 220522.770
  R² (Test): 0.814

Procesando modelo: XGBoost
Mejores parámetros: {'model__learning_rate': 0.2, 'model__max_depth': 7, 'model__n_estimators': 50}
Mejor MSE en CV: 233468234225.3256
Modelo: XGBoost
  MSE (CV): 233468234225.326
  MSE (Test): 342215183592.228
  MAE (Test): 233633.281
  R² (Test): 0.793

Procesando modelo: CatBoost
Mejores parámetros: {'model__depth': 7, 'model__iterations': 200, 'model__learning_rate': 0.1}
Mejor MSE en CV: 218755113803.01776
Modelo: CatBoost
  MSE (CV): 218755113803.018
  MSE (Test): 327029867412.354
  MAE (Te

In [21]:
 filepath = './Datos.csv'  # Asegúrate que este archivo esté en el mismo directorio o ajusta la ruta
df = load_data(filepath)
df.info()

df = df.drop(columns=['provincia', 'PrecioAnterior', 'descripcion', 'Enlace', 'titulo'])

# 2. Calcular la media del precio para cada zona
mean_prices = df.groupby('zona')['PrecioActual'].mean()

# Ordenar las zonas de menor a mayor media
sorted_zones = mean_prices.sort_values()

# Asignar un ranking (0 para la zona con la menor media, y así sucesivamente)
zona_ranking = {zona: rank for rank, zona in enumerate(sorted_zones.index)}

# Asignar el ranking como una nueva columna en el DataFrame
df['zona_rank'] = df['zona'].map(zona_ranking)
df = df.drop(columns=['zona'])

# 3. Procesar la columna "ascensor": mapear 'S' a 1 y 'N' a 0
df['ascensor'] = df['ascensor'].map({'S': 1, 'N': 0})

# 4. Procesar la columna "localizacion": rellenar valores nulos con una etiqueta "Desconocido"
df['localizacion'] = df['localizacion'].fillna('Desconocido')

# 5. Procesar la columna "planta": extraer un valor numérico
df['planta_processed'] = df['planta'].apply(process_planta)
df = df.drop(columns=['planta'])

# 6. Procesar la columna "tags": convertir la cadena separada por comas en variables dummy
df['tags'] = df['tags'].fillna("")
# Se convierte cada string en una lista de tags (limpiando espacios)
df['tags_list'] = df['tags'].apply(lambda x: [tag.strip() for tag in x.split(',')] if x != "" else [])
mlb = MultiLabelBinarizer()
tags_df = pd.DataFrame(mlb.fit_transform(df['tags_list']),
                        columns=['tag_' + t for t in mlb.classes_],
                        index=df.index)
# Se concatenan las columnas dummy al dataframe original
df = pd.concat([df, tags_df], axis=1)
df = df.drop(columns=['tags', 'tags_list'])

# 7. Codificar variables categóricas: "zona" y "localizacion"
# Se utiliza get_dummies y se evita la trampa de la multicolinealidad (drop_first=True)
df = pd.get_dummies(df, columns=['localizacion'], drop_first=True)

# 8. Manejar valores nulos: se puede optar por eliminar filas o rellenar con la media/mediana
# Aquí se opta por eliminar filas con valores nulos
df = df.dropna()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11826 entries, 0 to 11825
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   provincia       11826 non-null  object 
 1   zona            11826 non-null  object 
 2   titulo          11826 non-null  object 
 3   PrecioActual    11826 non-null  int64  
 4   PrecioAnterior  11826 non-null  int64  
 5   metros          11826 non-null  int64  
 6   habitaciones    11460 non-null  float64
 7   ascensor        11033 non-null  object 
 8   localizacion    10730 non-null  object 
 9   planta          10601 non-null  object 
 10  baños           11826 non-null  int64  
 11  tags            11664 non-null  object 
 12  descripcion     11761 non-null  object 
 13  Enlace          11826 non-null  object 
dtypes: float64(1), int64(4), object(9)
memory usage: 1.3+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 10204 entries, 0 to 11825
Data columns (total 66 columns):
 #   